# Team DeepRacer - Final Project
### <u>Alexis Wukich</u> | <u>Dipesh Pandya</u> | <u>Dhruba Chakrabarti</u>

### Installs
This cell installs all the different components that are required to run this project.

In [17]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install python-dotenv
!pip install langchain
!pip install langchain-google-genai
!pip install langchain_community
!pip install gradio
!pip install audiocraft # importing audiocraft to run facebook musicgen model
!pip install pydub # for converting wav file to mp3 and makiing longer sound clip
!pip install soundfile # exporting file in MP3

ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/operations/che

### Imports
This cell imports all the required packages.

In [18]:
import audiocraft
import gradio as gr
import moviepy.editor as mpy
from moviepy.editor import *
from moviepy.video.tools.segmenting import findObjects
import os
import soundfile as sf # for
import torch
import requests
import json
import pandas as pd

from audiocraft.models import MusicGen
from dotenv import load_dotenv
from langchain.chains import APIChain
from langchain_google_genai import ChatGoogleGenerativeAI
from pydub import AudioSegment
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification

In [19]:
# Import the pipeline class from the transformers module.
# from transformers import pipeline
# Instantiate the pipeline class for summarization using the facebook/bart-large-cnn model.
# from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
# import torch

#For music generation

#For prompt generation
# from transformers import AutoModelForCausalLM, AutoTokenizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

model = MusicGen.get_pretrained('facebook/musicgen-small') #importing facebook MusicGen Small model

prompt_model = AutoModelForCausalLM.from_pretrained("UnfilteredAI/Promt-generator")# using pretrainedprompt generator

emotion_model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-roberta-large")

prompt_tokenizer = AutoTokenizer.from_pretrained("UnfilteredAI/Promt-generator")

emotion_tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-roberta-large")

  warnings.warn(

  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")

  warnings.warn(



In [20]:
from google.colab import userdata
TMDB_API_KEY = userdata.get('TMDB_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"

In [21]:
# prompt Function
# def prompt(movie_title, emo_output):

#   prompt = f"Generate 10 seconds music clip for {movie_title} movie to express emotions full of {emo_output[1]} and bit of {emo_output[0]}"
#   #Music filled with {emo_output} for {movie_title}"
#   inputs = prompt_tokenizer(prompt, return_tensors="pt")
#   outputs = prompt_model.generate(**inputs, max_new_tokens=8) # restricting total new tokens to 8
#   generated_prompt = prompt_tokenizer.decode(outputs[0], skip_special_tokens=True) # extracting generated prompt to pass to next function

#   return generated_prompt

In [22]:
# get_movie_id_by_title Function

def get_movie_id_by_title(movie_title):

    print(f"Getting details for movie: {movie_title}")

    url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API_KEY}&query={movie_title}"

    # Send a request to TMDb API
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        search_results = response.json().get('results', [])

        #connect to google sheets file named movie genres on google drive
        genres_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQTIxZKTVbyc2DuRKw1iMe3Zk2BtIto2FyrLyffj0wmA5ilJDLiNGKgZQ2U5UAYDgmiAHrZ3tkAGbIW/pub?gid=0&single=true&output=csv")
        genres_df
        print("Getting Overview...")
        # If there are results, return the first movie's ID
        if search_results:
            movie = search_results[0]  # Get the first movie in the results
            movie_id = movie['id']
            title = movie['title']
            genre = movie['genre_ids']
            genre_name = genres_df[genres_df['Genre ID'].isin(genre)]['Genre Name'].tolist()
            genres = ', '.join(genre_name)
            generes_one = " & ".join(genre_name)
            overview = movie.get('overview', "No overview available")
            print("Getting keywords...")
            # Get keywords using movie id
            keywords_url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={TMDB_API_KEY}"
            keywords_response = requests.get(keywords_url)

            if keywords_response.status_code == 200:

                # Parse the JSON response to extract keywords
                keywords_data = keywords_response.json().get('keywords', [])

                # Extract the 'name' field for each keyword
                keywords = [keyword['name'] for keyword in keywords_data]

                #create a string
                keywords_str = ', '.join(keywords)

            else:
                keywords_str = "No keywords available"

            print("Getting images...")
            # Get images using movie id
            # url = f"https://api.themoviedb.org/3/movie/{movie_id}/images?api_key={TMDB_API_KEY}"
            url = f"https://api.themoviedb.org/3/movie/{movie_id}/images?language=en-US&include_image_language=en,null&api_key={TMDB_API_KEY}"
            image_response = requests.get(url)

            if image_response.status_code == 200:
                images_data = image_response.json()

                # Extract poster paths and backdrop paths
                poster_paths = [image['file_path'] for image in images_data.get('posters', [])]
                backdrop_paths = [image['file_path'] for image in images_data.get('backdrops', [])]

                #base_url
                base_url = "https://image.tmdb.org/t/p/"
                poster_size = "w300"
                backdrop_size = "w300"

                # image_urls = [f"{base_url}{size}{path}" for path in image_paths]
                poster_urls = [f"{base_url}{poster_size}{path}" for path in poster_paths]
                backdrop_urls = [f"{base_url}{backdrop_size}{path}" for path in backdrop_paths]

                # combine into one list

                image_urls = poster_urls + backdrop_urls

            else:
                print(f"Error: {image_response.status_code}")
                return None, None, None

            print("Generating emotions...")

            # Get emotions from keywords """Detect emotion using RoBERTa emotion classifier."""
            inputs = emotion_tokenizer(keywords_str, return_tensors="pt", truncation=True)
            outputs = emotion_model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            # Emotion labels according to the model's training
            emotion_labels = ["anger", "joy", "optimism", "sadness", "fear", "love", "surprise"]
            keyword_emotion = emotion_labels[probs.argmax()]

            # get emotions from overview
            inputs2 = emotion_tokenizer(overview, return_tensors="pt", truncation=True)
            outputs2 = emotion_model(**inputs2)
            probs2 = torch.nn.functional.softmax(outputs2.logits, dim=-1)
            # Emotion labels according to the model's training
            emotion_labels = ["anger", "joy", "optimism", "sadness", "fear", "love", "surprise"]
            overview_emotion = emotion_labels[probs2.argmax()]

            print(f"Emotions generated: {keyword_emotion} and {overview_emotion}")

            # combine emotions
            emotion = [keyword_emotion, overview_emotion]

            print("Selecting instruments...")

            # music prompt
            instruments = {'Instrument': ['violin', 'cello', 'bass', 'guitar', 'electric guitar', 'acoustic guitar', 'harp', 'drums', 'bongos', 'tabla', 'cymbals', 'hand claps', 'saxophone', 'flute', 'clarinet', 'trumpet', 'brass section', 'synthesizers', 'pads', 'bass synth', 'arpeggiator', 'modular synth', 'piano', 'electric piano', 'harpsichord', 'organ', 'electric bass', 'standup bass', 'bass guitar', 'choir', 'solo vocals', 'spoken word', 'vocal chops'],
                    'category': ['Strings', 'Strings', 'Strings', 'Strings', 'Strings', 'Strings', 'Strings', 'Percussion', 'Percussion', 'Percussion', 'Percussion', 'Percussion', 'Wind', 'Wind', 'Wind', 'Wind', 'Wind', 'Electronic', 'Electronic', 'Electronic', 'Electronic', 'Electronic', 'Piano and Keys', 'Piano and Keys', 'Piano and Keys', 'Piano and Keys', 'Bass', 'Bass', 'Bass', 'Vocal Elements', 'Vocal Elements', 'Vocal Elements', 'Vocal Elements']
                    }

            instrument_df = pd.DataFrame(instruments)

            # pick 3 random instruments from instrument_df
            Non_percussion = instrument_df[instrument_df['category'] != "Percussion"].sample(n=2)
            percussion = instrument_df[instrument_df['category'] == "Percussion"].sample(n=1)
            print (f"Instruments selected: {Non_percussion} and {percussion}")
            # join percussion and Non_percussion
            non_percussion_and_percussion = pd.concat([Non_percussion, percussion])
            joined_instruments = " & ".join(non_percussion_and_percussion['Instrument'])
            joined_instruments

            prompt = f"A track filled with {emotion[0]} and {emotion[1]} featuring {joined_instruments}, suitable for a movie project that has {generes_one} genres"
            print(f"Prompt generated: {prompt}")
        return {'movie_id': movie_id,
                'title': title,
                'overview': overview,
                'keywords': keywords_str,
                'image_urls': image_urls,
                'detected_emotions': emotion,
                'music_prompt': prompt,
        }
    else:
        print("No movie found with that title.")
        return None, None, None



In [23]:
def movie_music(movie_title, input_prompt):

  #define clip time
  model.set_generation_params(duration=15) # creating 10 seonds music. This can be created as variable in future
  #model.set_generation_params(duration=15)
  print("Generating music...")

  #generating music tensors using facebook model and prompt generated by emotions
  results_dp1 = model.generate([input_prompt]) # using generated prompt for music clip creation
  #sampling_rate = model.sample_rate

  #Extracting the generated music
  result_new = results_dp1[0][0]

  sampling_rate = 32000
  # sampling_rate = model.config.audio_encoder.sampling_rate
  #Audio(result_new.numpy(), rate=sampling_rate)

  #save the .wav file

  sf.write('output.wav', result_new.numpy(), sampling_rate)
  #convert .wave file to mp3
  import os
  from pydub import AudioSegment
  sound = AudioSegment.from_wav("output.wav")
  #creating mp3 file with music clip
  sound.export(f"{movie_title}.mp3", format="mp3")

  # Load the 15-second clip (example using pydub for WAV files)
  clip = AudioSegment.from_file(f"{movie_title}.mp3")

  # Split and rearrange segments

  # Rearrange and apply subtle fades
  extended_clip = clip * 6 # this can be defined as variable for future runs

  segment1 = extended_clip[:10000].fade_in(5000)   # First 10 seconds where it fades in at first 5 seonds
  #segment2 = extended_clip[10000:80000]  # balance seconds before last 10 seconds
  segment2 = extended_clip[10000:].fade_out(5000)  # Last 10 seconds where it starts fading out at last 5 seconds

  extended_clip_fifo = segment1 + segment2 # combining 3 segments to make one clip
  file_name = f"{movie_title}.mp3"
  extended_clip_fifo.export(file_name, format="mp3")
  return file_name

In [24]:
# movie_gen Function
def moviegen(images, music, moviename):
  print("Generating movie...")
  clips=[]
  for img_url in images:
    clip = ImageClip(img_url).set_duration(5)
    clips.append(clip)
  video_clip = concatenate_videoclips(clips, method="compose")
  audio_clip = mpy.AudioFileClip(music)
  video_clip = video_clip.set_audio(audio_clip)
  moviefilename=f"{moviename}.mp4"
  video_clip.write_videofile(moviefilename, fps=24)
  return moviefilename

In [25]:
# interface_fn Function
def interface_fn(movie_title):
    movie_details = get_movie_id_by_title(movie_title)

    if movie_details:
        image_output = movie_details['image_urls']
        emotion_output = movie_details['detected_emotions']

        # 1. use emotions to create prompt to generate music
        # 2. generate music
        # 3. determine number of images. if less than 20, combine list of image multiple times until >20 then subset from list.
        # 4. generate movie

        # 1.
        input_prompt = movie_details['music_prompt']

        # 2.
        music_file  = movie_music(movie_details['title'], input_prompt)

        # 3.
        image_files_first = movie_details['image_urls'][0]
        image_files_last = movie_details['image_urls'][-17:]
        image_files = [image_files_first] + image_files_last

        # 4.
        video_output = moviegen(image_files, music_file, movie_title)
        return emotion_output, image_output, input_prompt, video_output
    else:
        return "No movie found.", []

In [ ]:
app = gr.Interface(
    fn=interface_fn,
    inputs=gr.Textbox(label="What movie are you in the mood to watch?"),
    outputs=[gr.Textbox(label="Detected Emotion(s)"), gr.Textbox(label="Movie Images"), gr.Textbox(label="Prompt generated for music"), gr.Video(label="playable_video")],
    title="Movie Sentiment Analyzer",
    description="Share the movie you're watching to find the emotions and images to create your trailer."
)

# Launch the interface
app.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1648200ddd68299c02.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Getting details for movie: Ready Player One
Getting Overview...
Getting keywords...
Getting images...
Generating emotions...
Emotions generated: optimism and love
Selecting instruments...
Instruments selected:        Instrument    category
21  modular synth  Electronic
0          violin     Strings and   Instrument    category
9      tabla  Percussion
Prompt generated: A track filled with optimism and love featuring modular synth & violin & tabla, suitable for a movie project that has Adventure & Action & Science Fiction genres
Generating music...
Generating movie...
Moviepy - Building video Ready Player One.mp4.
MoviePy - Writing audio in Ready Player OneTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Ready Player One.mp4



Moviepy - Done !
Moviepy - video ready Ready Player One.mp4


In [26]:
# # test case - TEST MOVIE and search by name to get ID
# movie_title = "star wars"
# movie_details = get_movie_id_by_title(movie_title)

# # Print the movie ID, title, and overview
# if movie_details:
#     print(f"Movie ID: {movie_details['movie_id']}")
#     print(f"Title: {movie_details['title']}")
#     print(f"Overview: {movie_details['overview']}")
#     print(f"Keywords: {movie_details['keywords']}")
#     print(f"Image URLs: {movie_details['image_urls']}")
#     print(f"Detected Emotion(s): {movie_details['detected_emotions']}")
# else:
#     print("Movie not found.")